# Julia tries Linear Regression

## Imports

In [2]:
# Pandas library for the pandas dataframes
import pandas as pd    

# Import Scikit-Learn library for the regression models
import sklearn         
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# Note - you will need version 0.24.1 of scikit-learn to load this library (SequentialFeatureSelector)
from sklearn.feature_selection import f_regression, SequentialFeatureSelector
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Import numpy 
import numpy as np

# Another statistic model library
import statsmodels.api as sm
import statsmodels.formula.api as smf

import scipy.stats as stats
import scipy
from scipy import interpolate
from scipy.interpolate import interp1d

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt

# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 14})

# Command to automatically reload modules before executing cells
# not needed here but might be if you are writing your own library 
%load_ext autoreload
%autoreload 2

## Load data 7

In [7]:
data7 = pd.read_csv('mol_res_scan_results_7.csv')

In [8]:
data7.describe()

,Unnamed: 0,cut 1,cut 2,yield,purity,c_s,v,mol1_D_L,mol1_k_ov,mol1_q_max,...,mol1_z_p,mol1_n,mol1_K_s,mol2_D_L,mol2_k_ov,mol2_q_max,mol2_K_eq,mol2_z_p,mol2_n,mol2_K_s
count,16500.000000,16500.000000,16500.000000,16500.000000,16500.000000,1.650000e+04,16500.000000,1.650000e+04,1.650000e+04,16500.000000,...,16500.000000,16500.0,16500.0,1.650000e+04,1.650000e+04,16500.000000,16500.000000,16500.000000,16500.0,16500.0
mean,8249.500000,9.000000,21.000000,0.426965,0.795011,2.000000e-01,0.526270,5.900000e-02,5.420000e-01,72.426340,...,3.042140,0.0,0.0,5.900000e-02,5.420000e-01,73.707916,474.787868,2.968698,0.0,0.0
std,4763.284056,7.348692,7.348692,0.228462,0.048249,2.775642e-17,0.183799,2.775642e-17,1.110257e-16,43.183757,...,0.564033,0.0,0.0,2.775642e-17,1.110257e-16,44.486202,293.493286,0.588275,0.0,0.0
min,0.000000,0.000000,3.000000,0.031297,0.415009,2.000000e-01,0.211107,5.900000e-02,5.420000e-01,1.099948,...,2.006480,0.0,0.0,5.900000e-02,5.420000e-01,0.161089,7.957455,2.001373,0.0,0.0
25%,4124.750000,3.000000,15.000000,0.241406,0.766996,2.000000e-01,0.366209,5.900000e-02,5.420000e-01,32.822342,...,2.579930,0.0,0.0,5.900000e-02,5.420000e-01,34.389358,214.542791,2.486355,0.0,0.0
50%,8249.500000,9.000000,21.000000,0.401744,0.800083,2.000000e-01,0.524242,5.900000e-02,5.420000e-01,72.047476,...,3.064739,0.0,0.0,5.900000e-02,5.420000e-01,68.770202,443.062310,2.944435,0.0,0.0
75%,12374.250000,15.000000,27.000000,0.584216,0.825319,2.000000e-01,0.700572,5.900000e-02,5.420000e-01,110.047170,...,3.542654,0.0,0.0,5.900000e-02,5.420000e-01,117.539476,719.927531,3.494906,0.0,0.0
max,16499.000000,27.000000,30.000000,1.194263,0.960642,2.000000e-01,0.830669,5.900000e-02,5.420000e-01,145.621950,...,3.991703,0.0,0.0,5.900000e-02,5.420000e-01,145.422960,998.278730,3.995346,0.0,0.0


In [3]:
data7.head()

,Unnamed: 0,cut 1,cut 2,yield,purity,c_s,v,mol1_D_L,mol1_k_ov,mol1_q_max,...,mol1_z_p,mol1_n,mol1_K_s,mol2_D_L,mol2_k_ov,mol2_q_max,mol2_K_eq,mol2_z_p,mol2_n,mol2_K_s
0,0,0.0,3.0,0.242987,0.601903,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
1,1,0.0,6.0,0.340549,0.610753,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
2,2,0.0,9.0,0.401249,0.629828,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
3,3,0.0,12.0,0.450786,0.647608,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
4,4,0.0,15.0,0.493797,0.663042,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0


## Specify X columns as `X_columns`

In [4]:
column_names = data7.columns.values
print(column_names)
X_columns = column_names[5:21]
print('X_columns:')
print(X_columns)

['Unnamed: 0' 'cut 1' 'cut 2' 'yield' 'purity' 'c_s' 'v' 'mol1_D_L'
 'mol1_k_ov' 'mol1_q_max' 'mol1_K_eq' 'mol1_z_p' 'mol1_n' 'mol1_K_s'
 'mol2_D_L' 'mol2_k_ov' 'mol2_q_max' 'mol2_K_eq' 'mol2_z_p' 'mol2_n'
 'mol2_K_s']
X_columns:
['c_s' 'v' 'mol1_D_L' 'mol1_k_ov' 'mol1_q_max' 'mol1_K_eq' 'mol1_z_p'
 'mol1_n' 'mol1_K_s' 'mol2_D_L' 'mol2_k_ov' 'mol2_q_max' 'mol2_K_eq'
 'mol2_z_p' 'mol2_n' 'mol2_K_s']


## Observe yield regression

In [5]:
X = data7[X_columns].values
y_yield = data7[['yield']].values.reshape(-1, 1)
y_purity = data7[['purity']].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_yield, test_size=0.2, random_state=4, shuffle=True)

In [7]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the training data, the X values are times and the Y values are positions of the Cheetah
regr.fit(X_train, y_train)
beta1 = regr.coef_[0][0]
beta0 = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', beta1 , 'Intercept: ', beta0 )

# From the equation
Y_calc_test_2 = beta1*X_test + beta0

# Another way to get this is using the regr.predict function
Y_calc_test = regr.predict(X_test)

# Predict the values of  𝑦  in the test set using our fitted parameters.
Y_calc_test = regr.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))

# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

# OLS Regression
mreg = sm.OLS(y_train, X_train).fit()
mreg.summary(alpha=0.1) # Set significance level

Scikit learn - Slope:  52000128.20057324 Intercept:  427435425.6673447
Mean squared error: 0.04
Coefficient of determination: 0.25


/home/j19a9k3/miniconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     621.2
Date:                Tue, 27 Apr 2021   Prob (F-statistic):               0.00
Time:                        09:01:29   Log-Likelihood:                 2638.4
No. Observations:               13200   AIC:                            -5261.
Df Residuals:                   13192   BIC:                            -5201.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|       [0.05       0.95]
------------------------------------------------------------------------------
x1             0.0528      0.005     10.544      0.000       0.045       0.061
x2             0.4720      0.010     49.667      0.000       0.456       0.488
x3             0.0156      0.001     10.544      0.000       0.013       0.018
x4             0.1431      0.014     10.544      0.000       0.121       0.165
x5          1.867e-05   4.02e-05      0.465      0.642   -4.74e-05    8.47e-05
x6            -0.0003   6.21e-06    -41.021      0.000      -0.000      -0.000
x7             0.0428      0.003     13.954      0.000       0.038       0.048
const      -8.502e-17   3.95e-18    -21.500      0.000   -9.15e-17   -7.85e-17
x8                  0          0        nan        nan           0           0
x9             0.0156      0.001     10.544      0.000       0.013       0.018
x10            0.1431      0.014     10.544      0.000       0.121       0.165
x11          4.27e-05   3.93e-05      1.086      0.277   -2.19e-05       0.000
x12        -9.257e-06   5.96e-06     -1.553      0.120   -1.91e-05    5.49e-07
x13            0.0035      0.003      1.164      0.244      -0.001       0.008
x14                 0          0        nan        nan           0           0
x15                 0          0        nan        nan           0           0
==============================================================================
Omnibus:                      619.839   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              267.329
Skew:                          -0.099   Prob(JB):                     8.92e-59
Kurtosis:                       2.331   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Select features that are most important

In [18]:
# Find features which are most important
sfs_forward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='forward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_forward.get_support(),"\n")
selected = sfs_forward.get_support(indices=True)
print("Selected input features using Forward Stepwise Selection:\n", f_names[selected])

[False  True False False False  True  True False False False False False
 False False False False] 

Selected input features using Forward Stepwise Selection:
 ['v' 'mol1_K_eq' 'mol1_z_p']


In [19]:
sfs_backward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='backward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_backward.get_support(),"\n")
selected = sfs_backward.get_support(indices=True)
print("Selected input features using Backward Stepwise Selection:\n", f_names[selected])

[False  True False False False  True  True False False False False False
 False False False False] 

Selected input features using Backward Stepwise Selection:
 ['v' 'mol1_K_eq' 'mol1_z_p']


## Observe purity regression

In [8]:
X = data7[X_columns].values
y_yield = data7[['yield']].values.reshape(-1, 1)
y_purity = data7[['purity']].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_purity, test_size=0.2, random_state=4, shuffle=True)

In [10]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the training data, the X values are times and the Y values are positions of the Cheetah
regr.fit(X_train, y_train)
beta1 = regr.coef_[0][0]
beta0 = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', beta1 , 'Intercept: ', beta0 )

# From the equation
Y_calc_test_2 = beta1*X_test + beta0

# Another way to get this is using the regr.predict function
Y_calc_test = regr.predict(X_test)

# Predict the values of  𝑦  in the test set using our fitted parameters.
Y_calc_test = regr.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))

# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

# OLS Regression
mreg = sm.OLS(y_train, X_train).fit()
mreg.summary(alpha=0.1) # Set significance level

Scikit learn - Slope:  -2026606.8943159017 Intercept:  -16658488.53025712
Mean squared error: 0.00
Coefficient of determination: 0.79


/home/j19a9k3/miniconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     7321.
Date:                Tue, 27 Apr 2021   Prob (F-statistic):               0.00
Time:                        09:04:07   Log-Likelihood:                 31688.
No. Observations:               13200   AIC:                        -6.336e+04
Df Residuals:                   13192   BIC:                        -6.330e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|       [0.05       0.95]
------------------------------------------------------------------------------
x1             0.2485      0.001    448.041      0.000       0.248       0.249
x2             0.0066      0.001      6.304      0.000       0.005       0.008
x3             0.0733      0.000    448.041      0.000       0.073       0.074
x4             0.6734      0.002    448.041      0.000       0.671       0.676
x5          3.988e-06   4.45e-06      0.897      0.370   -3.33e-06    1.13e-05
x6         -9.885e-05   6.87e-07   -143.858      0.000     -0.0001   -9.77e-05
x7             0.0185      0.000     54.360      0.000       0.018       0.019
const        1.29e-16   4.38e-19    294.653      0.000    1.28e-16     1.3e-16
x8                  0          0        nan        nan           0           0
x9             0.0733      0.000    448.041      0.000       0.073       0.074
x10            0.6734      0.002    448.041      0.000       0.671       0.676
x11        -1.179e-05   4.35e-06     -2.710      0.007   -1.89e-05   -4.63e-06
x12         9.892e-05    6.6e-07    149.873      0.000    9.78e-05       0.000
x13           -0.0166      0.000    -49.778      0.000      -0.017      -0.016
x14                 0          0        nan        nan           0           0
x15                 0          0        nan        nan           0           0
==============================================================================
Omnibus:                     6686.982   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           114379.801
Skew:                          -2.028   Prob(JB):                         0.00
Kurtosis:                      16.839   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Select features that are most important

In [18]:
# Find features which are most important
sfs_forward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='forward').fit(X, y_purity)

f_names = np.array(X_columns)
print(sfs_forward.get_support(),"\n")
selected = sfs_forward.get_support(indices=True)
print("Selected input features using Forward Stepwise Selection:\n", f_names[selected])


sfs_backward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='backward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_backward.get_support(),"\n")
selected = sfs_backward.get_support(indices=True)
print("Selected input features using Backward Stepwise Selection:\n", f_names[selected])

[False False False False False  True False False False False False  True
  True False False False] 

Selected input features using Forward Stepwise Selection:
 ['mol1_K_eq' 'mol2_q_max' 'mol2_K_eq']
[False  True False False  True  True False False False False False False
 False False False False] 

Selected input features using Backward Stepwise Selection:
 ['v' 'mol1_q_max' 'mol1_K_eq']


## Compare to Langmuir adsorption model

### Load data 8

In [13]:
data8 = pd.read_csv('mol_res_scan_results_8.csv')

### Regression for yield

In [14]:
X = data8[X_columns].values
y_yield = data8[['yield']].values.reshape(-1, 1)
y_purity = data8[['purity']].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_yield, test_size=0.2, random_state=4, shuffle=True)

# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the training data, the X values are times and the Y values are positions of the Cheetah
regr.fit(X_train, y_train)
beta1 = regr.coef_[0][0]
beta0 = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', beta1 , 'Intercept: ', beta0 )

# From the equation
Y_calc_test_2 = beta1*X_test + beta0

# Another way to get this is using the regr.predict function
Y_calc_test = regr.predict(X_test)

# Predict the values of  𝑦  in the test set using our fitted parameters.
Y_calc_test = regr.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))

# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

# OLS Regression
mreg = sm.OLS(y_train, X_train).fit()
mreg.summary(alpha=0.1) # Set significance level

Scikit learn - Slope:  53832192.663763754 Intercept:  444213441.24499047
Mean squared error: 0.01
Coefficient of determination: 0.42


/home/j19a9k3/miniconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     1355.
Date:                Tue, 27 Apr 2021   Prob (F-statistic):               0.00
Time:                        09:15:47   Log-Likelihood:                 10924.
No. Observations:               13200   AIC:                        -2.183e+04
Df Residuals:                   13192   BIC:                        -2.177e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|       [0.05       0.95]
------------------------------------------------------------------------------
x1            -0.0105      0.003     -4.164      0.000      -0.015      -0.006
x2             0.4964      0.005     94.736      0.000       0.488       0.505
x3            -0.0031      0.001     -4.164      0.000      -0.004      -0.002
x4            -0.0284      0.007     -4.164      0.000      -0.040      -0.017
x5            -0.0003   2.14e-05    -12.208      0.000      -0.000      -0.000
x6         -5.391e-05   3.14e-06    -17.182      0.000   -5.91e-05   -4.88e-05
x7            -0.0070      0.002     -4.253      0.000      -0.010      -0.004
const      -3.059e-17   1.16e-18    -26.405      0.000   -3.25e-17   -2.87e-17
x8                  0          0        nan        nan           0           0
x9            -0.0031      0.001     -4.164      0.000      -0.004      -0.002
x10           -0.0284      0.007     -4.164      0.000      -0.040      -0.017
x11        -4.886e-05   2.24e-05     -2.183      0.029   -8.57e-05    -1.2e-05
x12         2.051e-06   3.16e-06      0.650      0.516   -3.14e-06    7.24e-06
x13            0.0061      0.002      3.738      0.000       0.003       0.009
x14                 0          0        nan        nan           0           0
x15                 0          0        nan        nan           0           0
==============================================================================
Omnibus:                     2387.596   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10342.772
Skew:                           0.831   Prob(JB):                         0.00
Kurtosis:                       7.005   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Select features that are most important

In [15]:
# Find features which are most important
sfs_forward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='forward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_forward.get_support(),"\n")
selected = sfs_forward.get_support(indices=True)
print("Selected input features using Forward Stepwise Selection:\n", f_names[selected])


sfs_backward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='backward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_backward.get_support(),"\n")
selected = sfs_backward.get_support(indices=True)
print("Selected input features using Backward Stepwise Selection:\n", f_names[selected])

[False  True False False  True  True False False False False False False
 False False False False] 

Selected input features using Forward Stepwise Selection:
 ['v' 'mol1_q_max' 'mol1_K_eq']
[False  True False False  True  True False False False False False False
 False False False False] 

Selected input features using Backward Stepwise Selection:
 ['v' 'mol1_q_max' 'mol1_K_eq']


## Regression for purity

In [16]:
X = data8[X_columns].values
y_yield = data8[['yield']].values.reshape(-1, 1)
y_purity = data8[['purity']].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_purity, test_size=0.2, random_state=4, shuffle=True)

# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the training data, the X values are times and the Y values are positions of the Cheetah
regr.fit(X_train, y_train)
beta1 = regr.coef_[0][0]
beta0 = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', beta1 , 'Intercept: ', beta0 )

# From the equation
Y_calc_test_2 = beta1*X_test + beta0

# Another way to get this is using the regr.predict function
Y_calc_test = regr.predict(X_test)

# Predict the values of  𝑦  in the test set using our fitted parameters.
Y_calc_test = regr.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))

# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

# OLS Regression
mreg = sm.OLS(y_train, X_train).fit()
mreg.summary(alpha=0.1) # Set significance level

Scikit learn - Slope:  52368159.933254026 Intercept:  432132510.86383456
Mean squared error: 0.00
Coefficient of determination: 0.24


/home/j19a9k3/miniconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     553.7
Date:                Tue, 27 Apr 2021   Prob (F-statistic):               0.00
Time:                        09:17:25   Log-Likelihood:                 22700.
No. Observations:               13200   AIC:                        -4.538e+04
Df Residuals:                   13192   BIC:                        -4.532e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|       [0.05       0.95]
------------------------------------------------------------------------------
x1             0.2561      0.001    247.942      0.000       0.254       0.258
x2             0.0063      0.002      2.913      0.004       0.003       0.010
x3             0.0756      0.000    247.942      0.000       0.075       0.076
x4             0.6940      0.003    247.942      0.000       0.689       0.699
x5            -0.0002   8.77e-06    -24.728      0.000      -0.000      -0.000
x6         -4.019e-05   1.29e-06    -31.255      0.000   -4.23e-05   -3.81e-05
x7            -0.0101      0.001    -14.840      0.000      -0.011      -0.009
const       1.025e-16   4.75e-19    215.837      0.000    1.02e-16    1.03e-16
x8                  0          0        nan        nan           0           0
x9             0.0756      0.000    247.942      0.000       0.075       0.076
x10            0.6940      0.003    247.942      0.000       0.689       0.699
x11            0.0003   9.17e-06     28.317      0.000       0.000       0.000
x12         3.635e-05   1.29e-06     28.106      0.000    3.42e-05    3.85e-05
x13            0.0036      0.001      5.401      0.000       0.002       0.005
x14                 0          0        nan        nan           0           0
x15                 0          0        nan        nan           0           0
==============================================================================
Omnibus:                    10377.391   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           655553.371
Skew:                          -3.279   Prob(JB):                         0.00
Kurtosis:                      36.896   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [17]:
# Find features which are most important
sfs_forward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='forward').fit(X, y_purity)

f_names = np.array(X_columns)
print(sfs_forward.get_support(),"\n")
selected = sfs_forward.get_support(indices=True)
print("Selected input features using Forward Stepwise Selection:\n", f_names[selected])


sfs_backward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=3,
                                        direction='backward').fit(X, y_yield)

f_names = np.array(X_columns)
print(sfs_backward.get_support(),"\n")
selected = sfs_backward.get_support(indices=True)
print("Selected input features using Backward Stepwise Selection:\n", f_names[selected])

[False False False False False  True False False False False False  True
  True False False False] 

Selected input features using Forward Stepwise Selection:
 ['mol1_K_eq' 'mol2_q_max' 'mol2_K_eq']
[False  True False False  True  True False False False False False False
 False False False False] 

Selected input features using Backward Stepwise Selection:
 ['v' 'mol1_q_max' 'mol1_K_eq']
